**The dataset** is collected from UCI Machine Learning Repository through the following [link](https://archive.ics.uci.edu/ml/datasets/Unmanned+Aerial+Vehicle+%28UAV%29+Intrusion+Detection)

This application is working in first dataset (Bidirectional-flow/Parrot Bebop1), combined first dataset can be [downloaded](http://mason.gmu.edu/~lzhao9/materials/data/UAV/data/pub_dataset1.mat) from Liang Zhao homepage.Bidirectional-flow mode will involve 9 features × 2 sources × 3 direction flow = 54 features for more info visit this [link](http://mason.gmu.edu/~lzhao9/materials/data/UAV/)

extract data with its default name `pub_dataset1.mat` in `__data__` directory

In [14]:
import numpy as np
import pandas as pd
import h5py

import matplotlib.pyplot as plt
import seaborn as sns

from pprint import pprint

In [36]:
# use HDF reader for matlab v7.3 files
data = { k:np.array(v).T for k, v in h5py.File('./__data__/pub_dataset1.mat').items()}
data.keys()

dict_keys(['D', 'H', 'data_te', 'data_tr'])

$n$ is the number of training samples   
$k$ is the number of feature   
$n^{\prime}$ is the number of testing samples    
$k^{\prime}$ is the number of feature computational components and k is the numbe of features.  
The last column of `data_te` and `data_tr` is the label: `1 means UAV, 0 otherwise`

--- 
$\text{data_tr} \in R^{n×(k+1)}$   
$\text{data_te} \in R^{n^{\prime}×(k+1)}$   
$D \in R^{k×1}$. The generation runtime for each feature.  
$H \in R^{k^{\prime}×k}$. The incident matrix of the feature computational hypergraph (see the paper for details). 


In [55]:
def reset_random_seed(seed=1917):
    np.random.seed(seed)

In [48]:
X = data['data_tr'][:, :-1]
y = data['data_tr'][:, -1]

X_test = data['data_te'][:, :-1]
y_test = data['data_te'][:, -1]

## Accuracy Optimization

- MLP
- AutoEncoder
- Proposed Method

In [68]:
from sklearn.neural_network import MLPClassifier

In [65]:
reset_random_seed()
model = MLPClassifier()
model.fit(X, y)
model.score(X_test, y_test)

0.9931930342050032

Auto Encoder
- see the slides 
- see this link https://i-systems.github.io/teaching/ML/iNotes/15_Autoencoder.html

proposed method will be the follwoing  
- classify   
- print wrong data   
- train model on wrong data such as svm  
- use svm for that classified datas
- o.w use mlp normal model

## Runtime Optimization

- TODO: Optimize run Time